# Chroma
### Open Source Vector-DB
<img src="img/chroma.png" width="600"/>  
  
Chroma è un progetto Open Source nato per gestire il modo in cui vengono rappresentati i dati in Intelligenza Artificiale: i vettori di Embeddings.  
Permette di creare e ricercare su vettori di incorporamento che rappresentano testi e immagni, presto verrà integrata la possibilità di lavorare anche su audio e video.  
È possibile utilizzare i modelli di Embeddings messi a disposizione dai principali provider, come anche utilizzare modelli in locale o funzioni custom per l'incorporamento delle informazioni in vettori numerici.  
  
Il modello utilizzato di default è `all-MiniLM-L6-v2` di [Sentence Transformers](https://www.sbert.net/) (a.k.a. SBERT), un modello locale che viene eventualmente scaricato automaticamente al suo primo utilizzo.    


In [2]:
import chromadb  # pip install chromadb
from chromadb.config import Settings

chroma_client = chromadb.Client(
    Settings(
        persist_directory="/db",
        allow_reset=True,
    )
)

In [ ]:
collection = chroma_client.create_collection(name="data_collection")

**Importante**: il nome di una Collection di Chroma è utilizzato da Chroma per inidicizzare le informazioni contenute e deve sottostare alle seguenti regole:
* da 3 a 63 caratteri
* deve iniziare e finire con una lettera minuscola o un numero
* può contenere all'interno solo questi caratteri speciali: `.-_`
* non può contenere due punti consecutivi
* il nome non deve essere un indirizzo IP valido

In [ ]:
doc_1_anagrafica = "Veronica Rossi è nata il 15 aprile 1997 a Bologna, Italia. Alta 1.70 m, ha capelli castani e occhi verdi. Attualmente vive a Milano, dove si è trasferita per motivi di studio e lavoro."
doc_1_studi = "Dopo aver ottenuto il diploma di maturità scientifica presso il Liceo Galileo Galilei di Bologna con il massimo dei voti, Veronica ha proseguito i suoi studi all'Università di Milano, dove si è laureata in Ingegneria Biomedica. Durante il suo percorso universitario, ha partecipato a un programma di scambio con il Massachusetts Institute of Technology (MIT) negli Stati Uniti, che le ha permesso di lavorare su progetti di ricerca avanzati nel campo delle neuroprotesi. Attualmente, è iscritta al dottorato di ricerca in Tecnologie Biomediche a Milano, dove si concentra sullo sviluppo di dispositivi medici innovativi."
doc_1_hobby = "Veronica ha una grande passione per la musica e suona il pianoforte da quando aveva sei anni. Partecipa regolarmente a concerti e competizioni, spesso vincendo premi per le sue performance. Oltre alla musica, ama l'escursionismo e trascorre gran parte del suo tempo libero esplorando i sentieri delle Alpi e degli Appennini. È anche una lettrice appassionata, con un interesse particolare per la letteratura classica e moderna italiana e straniera."

collection.add(
    documents = [doc_1_anagrafica, doc_1_studi, doc_1_hobby],
    metadatas = [{"tipo_doc": "anagrafica"}, {"tipo_doc": "studi"}, {"tipo_doc": "hobby"}],
    ids = ["id1", "id2", "id3"]
)

In [ ]:
collection.count()

3

In [ ]:
collection.get()

{'ids': ['id1', 'id2', 'id3'],
 'embeddings': None,
 'documents': ['Veronica Rossi è nata il 15 aprile 1997 a Bologna, Italia. Alta 1.70 m, ha capelli castani e occhi verdi. Attualmente vive a Milano, dove si è trasferita per motivi di studio e lavoro.',
  "Dopo aver ottenuto il diploma di maturità scientifica presso il Liceo Galileo Galilei di Bologna con il massimo dei voti, Veronica ha proseguito i suoi studi all'Università di Milano, dove si è laureata in Ingegneria Biomedica. Durante il suo percorso universitario, ha partecipato a un programma di scambio con il Massachusetts Institute of Technology (MIT) negli Stati Uniti, che le ha permesso di lavorare su progetti di ricerca avanzati nel campo delle neuroprotesi. Attualmente, è iscritta al dottorato di ricerca in Tecnologie Biomediche a Milano, dove si concentra sullo sviluppo di dispositivi medici innovativi.",
  "Veronica ha una grande passione per la musica e suona il pianoforte da quando aveva sei anni. Partecipa regolarmente

In [ ]:
collection.get("id2")

{'ids': ['id2'],
 'embeddings': None,
 'documents': ["Dopo aver ottenuto il diploma di maturità scientifica presso il Liceo Galileo Galilei di Bologna con il massimo dei voti, Veronica ha proseguito i suoi studi all'Università di Milano, dove si è laureata in Ingegneria Biomedica. Durante il suo percorso universitario, ha partecipato a un programma di scambio con il Massachusetts Institute of Technology (MIT) negli Stati Uniti, che le ha permesso di lavorare su progetti di ricerca avanzati nel campo delle neuroprotesi. Attualmente, è iscritta al dottorato di ricerca in Tecnologie Biomediche a Milano, dove si concentra sullo sviluppo di dispositivi medici innovativi."],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'tipo_doc': 'studi'}]}

In [ ]:
chroma_client.list_collections()

[Collection(name=data_collection)]

In [ ]:
collection.modify(name="miei_dati")

chroma_client.list_collections()

[Collection(name=miei_dati)]

In [ ]:
chroma_client.get_collection(name="miei_dati")

Collection(name=miei_dati)

In [ ]:
chroma_client.delete_collection(name="miei_dati")

chroma_client.list_collections()

[]

In [ ]:
chroma_client.reset()

chroma_client.list_collections()

[]

**La scelta della metrica di distanza nello spazio degli embeddings influenza profondamente il modo in cui valutiamo la somiglianza tra testi o concetti. Ecco una panoramica delle principali metriche e dei contesti d’uso più adatti:**

<img src="img/chroma_distances.png">

**• Squared L2 (Distanza Euclidea al Quadrato)**
Questa metrica misura la distanza "fisica" tra due vettori, tenendo conto sia della lunghezza sia della direzione. In pratica, più due punti sono vicini nello spazio, più sono considerati simili.
*Quando usarla*: è la scelta classica per applicazioni di clustering o quando vuoi raggruppare insiemi di testi che si distribuiscono in modo compatto e la scala dei vettori ha un significato. Se, ad esempio, vuoi raggruppare documenti che trattano argomenti simili e gli embeddings sono “calibrati” in termini di magnitudine, questa distanza è efficace.

**• Inner Product (Prodotto Interno / Scalare)**
Calcola quanto due vettori "puntano" nella stessa direzione, ma tiene conto anche della loro lunghezza. Non è propriamente una distanza, ma una misura di “affinità”.
*Quando usarla*: utile in sistemi di ranking e retrieval, dove l’obiettivo è ordinare risultati rispetto a una query (es. question answering). Se hai embeddings normalizzati (cioè di norma fissata), prodotto interno e cosine similarity coincidono. Se invece vuoi premiare anche embeddings “più intensi” (es. una risposta molto pertinente e ben rappresentata), il prodotto interno è la metrica giusta.

**• Cosine Similarity (Somiglianza Coseno)**
Questa metrica valuta solo la direzione tra i vettori, ignorando completamente la lunghezza. Il risultato è compreso tra -1 (opposti), 0 (ortogonali) e 1 (identici come direzione).
*Quando usarla*: ideale per confrontare quanto due testi (o parole) siano simili in termini di significato, a prescindere dall’intensità o dalla “forza” del vettore. È la scelta di default per la maggior parte delle applicazioni NLP perché gli embeddings dei LLM possono variare molto in lunghezza, ma la direzione codifica la semantica. Perfetta quando ti interessa sapere “quanto sono semanticamente allineati” due testi, indipendentemente dalla loro lunghezza.

---

**In sintesi:**

* Usa **Squared L2** se ti interessa la distanza “assoluta” tra punti, ad esempio per clustering.
* Scegli **Inner Product** se vuoi ordinare risultati in base alla “rilevanza complessiva” rispetto a una query.
* Preferisci **Cosine Similarity** se vuoi valutare la somiglianza semantica pura, ignorando la lunghezza degli embeddings.

In [ ]:
collection = chroma_client.create_collection(
    name="data_collection",
    metadata={"hnsw:space": "l2"}  # default
                                   # HNSW: È un algoritmo di ricerca approssimato
                                   # che consente di trovare i vettori più vicini
                                   # (Nearest Neighbors) in uno spazio ad alta dimensione
)

doc_1_anagrafica = "Veronica Rossi è nata il 15 aprile 1997 a Bologna, Italia. Alta 1.70 m, ha capelli castani e occhi verdi. Attualmente vive a Milano, dove si è trasferita per motivi di studio e lavoro."
doc_1_studi = "Dopo aver ottenuto il diploma di maturità scientifica presso il Liceo Galileo Galilei di Bologna con il massimo dei voti, Veronica ha proseguito i suoi studi all'Università di Milano, dove si è laureata in Ingegneria Biomedica. Durante il suo percorso universitario, ha partecipato a un programma di scambio con il Massachusetts Institute of Technology (MIT) negli Stati Uniti, che le ha permesso di lavorare su progetti di ricerca avanzati nel campo delle neuroprotesi. Attualmente, è iscritta al dottorato di ricerca in Tecnologie Biomediche a Milano, dove si concentra sullo sviluppo di dispositivi medici innovativi."
doc_1_hobby = "Veronica ha una grande passione per la musica e suona il pianoforte da quando aveva sei anni. Partecipa regolarmente a concerti e competizioni, spesso vincendo premi per le sue performance. Oltre alla musica, ama l'escursionismo e trascorre gran parte del suo tempo libero esplorando i sentieri delle Alpi e degli Appennini. È anche una lettrice appassionata, con un interesse particolare per la letteratura classica e moderna italiana e straniera."

collection.add(
    documents = [doc_1_anagrafica, doc_1_studi, doc_1_hobby],
    metadatas = [{"tipo_doc": "anagrafica"}, {"tipo_doc": "studi"}, {"tipo_doc": "hobby"}],
    ids = ["id1", "id2", "id3"]
)

In [ ]:
results = collection.query(
    query_texts=["qual'è il nome?"],
    n_results=3
)

results

{'ids': [['id1', 'id3', 'id2']],
 'embeddings': None,
 'documents': [['Veronica Rossi è nata il 15 aprile 1997 a Bologna, Italia. Alta 1.70 m, ha capelli castani e occhi verdi. Attualmente vive a Milano, dove si è trasferita per motivi di studio e lavoro.',
   "Veronica ha una grande passione per la musica e suona il pianoforte da quando aveva sei anni. Partecipa regolarmente a concerti e competizioni, spesso vincendo premi per le sue performance. Oltre alla musica, ama l'escursionismo e trascorre gran parte del suo tempo libero esplorando i sentieri delle Alpi e degli Appennini. È anche una lettrice appassionata, con un interesse particolare per la letteratura classica e moderna italiana e straniera.",
   "Dopo aver ottenuto il diploma di maturità scientifica presso il Liceo Galileo Galilei di Bologna con il massimo dei voti, Veronica ha proseguito i suoi studi all'Università di Milano, dove si è laureata in Ingegneria Biomedica. Durante il suo percorso universitario, ha partecipato a

In [ ]:
collection.update(
    ids=["id1"],
    documents=["Martina Rossi è nata il 15 aprile 1997 a Bologna, Italia. Alta 1.70 m, ha capelli castani e occhi verdi. Attualmente vive a Milano, dove si è trasferita per motivi di studio e lavoro."],
    #metadatas=[""],
)

In [ ]:
collection.get()

{'ids': ['id1', 'id2', 'id3'],
 'embeddings': None,
 'documents': ['Martina Rossi è nata il 15 aprile 1997 a Bologna, Italia. Alta 1.70 m, ha capelli castani e occhi verdi. Attualmente vive a Milano, dove si è trasferita per motivi di studio e lavoro.',
  "Dopo aver ottenuto il diploma di maturità scientifica presso il Liceo Galileo Galilei di Bologna con il massimo dei voti, Veronica ha proseguito i suoi studi all'Università di Milano, dove si è laureata in Ingegneria Biomedica. Durante il suo percorso universitario, ha partecipato a un programma di scambio con il Massachusetts Institute of Technology (MIT) negli Stati Uniti, che le ha permesso di lavorare su progetti di ricerca avanzati nel campo delle neuroprotesi. Attualmente, è iscritta al dottorato di ricerca in Tecnologie Biomediche a Milano, dove si concentra sullo sviluppo di dispositivi medici innovativi.",
  "Veronica ha una grande passione per la musica e suona il pianoforte da quando aveva sei anni. Partecipa regolarmente 

In [ ]:
results = collection.query(
    query_texts=["qual'è il nome?"],
    n_results=1
)

results

{'ids': [['id1']],
 'embeddings': None,
 'documents': [['Martina Rossi è nata il 15 aprile 1997 a Bologna, Italia. Alta 1.70 m, ha capelli castani e occhi verdi. Attualmente vive a Milano, dove si è trasferita per motivi di studio e lavoro.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'tipo_doc': 'anagrafica'}]],
 'distances': [[1.2315267324447632]]}

In [ ]:
collection.delete(ids = ['id1'])

results = collection.query(
    query_texts=["qual'è il nome?"],
    n_results=1
)

results

{'ids': [['id3']],
 'embeddings': None,
 'documents': [["Veronica ha una grande passione per la musica e suona il pianoforte da quando aveva sei anni. Partecipa regolarmente a concerti e competizioni, spesso vincendo premi per le sue performance. Oltre alla musica, ama l'escursionismo e trascorre gran parte del suo tempo libero esplorando i sentieri delle Alpi e degli Appennini. È anche una lettrice appassionata, con un interesse particolare per la letteratura classica e moderna italiana e straniera."]],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'tipo_doc': 'hobby'}]],
 'distances': [[1.2836096286773682]]}

In [ ]:
collection.count()

2

In [ ]:
collection.add(
    documents = [doc_1_anagrafica],
    metadatas = [{"tipo_doc": "anagrafica"}],
    ids = ["id1"]
)

In [ ]:
doc_2_anagrafica = "Maria Neri è nata il 23 agosto 1998 a Napoli, Italia. È alta 1.65 m, ha capelli neri lunghi e occhi marroni. Attualmente risiede a Torino, dove si è trasferita per continuare i suoi studi e la sua carriera professionale."
doc_2_studi = "Maria ha completato il suo diploma di liceo classico presso il Liceo Aristotele di Napoli con eccellenti risultati. Successivamente, si è iscritta all'Università di Torino, dove ha conseguito una laurea in Psicologia Clinica. Durante la sua carriera universitaria, ha partecipato a un programma di scambio accademico con l'Università di Cambridge nel Regno Unito, dove ha approfondito studi sui disturbi comportamentali infantili. Attualmente è iscritta a un Master in Neuropsicologia, focalizzandosi sulla ricerca e terapia dei disturbi cognitivi."
doc_2_hobby = "Maria è una appassionata artista, specializzata in pittura ad olio e acquarello. Partecipa spesso a mostre d'arte locali e internazionali, dove ha ricevuto diversi riconoscimenti per il suo lavoro. Ama anche viaggiare e ha visitato numerosi paesi, documentando i suoi viaggi attraverso la fotografia. Un altro suo grande interesse è la cucina; ama sperimentare ricette da diverse culture, con un particolare interesse per la cucina mediterranea e asiatica."

collection.add(
    documents = [doc_2_anagrafica, doc_2_studi, doc_2_hobby],
    metadatas = [{"tipo_doc": "anagrafica"},{"tipo_doc": "studi"},{'tipo_doc':'hobby'}],
    ids = ["id4", "id5", "id6"]
)


doc_3_anagrafica = "Luca Verdi è nato il 12 gennaio 1996 a Firenze, Italia. È alto 1.82 m, ha i capelli corti biondi e occhi azzurri. Attualmente vive a Roma, dove si è trasferito per lavoro dopo aver completato i suoi studi universitari."
doc_3_studi = "Luca ha ottenuto il diploma di liceo linguistico presso il Liceo Dante Alighieri di Firenze, dimostrando un'ottima padronanza delle lingue straniere. Successivamente, si è iscritto all'Università di Roma La Sapienza, dove si è laureato in Relazioni Internazionali. Durante gli anni universitari, ha trascorso un semestre a Berlino in un programma Erasmus, che gli ha permesso di approfondire la sua conoscenza della politica europea e internazionale. Dopo la laurea, ha completato un master in Diplomazia presso lo stesso ateneo, preparandosi per una carriera nel servizio diplomatico italiano."
doc_3_hobby = "Luca ha una passione per lo sport, praticando calcio e tennis fin dalla giovane età. È membro attivo di un club di calcio locale a Roma e partecipa a tornei amatoriali di tennis durante il fine settimana. Oltre allo sport, Luca è un avido lettore di libri di storia e politica, e si dedica al volontariato, collaborando con organizzazioni che promuovono la pace e lo sviluppo internazionale. Ha anche un interesse per la fotografia, che pratica principalmente durante i suoi viaggi all'estero."

collection.add(
    documents = [doc_3_anagrafica, doc_3_studi, doc_3_hobby],
    metadatas = [{"tipo_doc": "anagrafica"},{"tipo_doc": "studi"},{'tipo_doc':'hobby'}],
    ids = ["id7", "id8", "id9"]
)

In [ ]:
results = collection.query(
    query_texts=["chi è vissuto a Milano?"],
    n_results=1
)

results

{'ids': [['id1']],
 'embeddings': None,
 'documents': [['Veronica Rossi è nata il 15 aprile 1997 a Bologna, Italia. Alta 1.70 m, ha capelli castani e occhi verdi. Attualmente vive a Milano, dove si è trasferita per motivi di studio e lavoro.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'tipo_doc': 'anagrafica'}]],
 'distances': [[0.8363277316093445]]}

In [ ]:
# da notare che ora evidenzierà documenti in cui non
# c'è "Milano" ma comunque una città, trovandoli
# comunque come documenti vicini all'informazione
# richiesta

collection.delete(ids = ['id1'])

results = collection.query(
    query_texts=["chi è vissuto a Milano?"],
    n_results=5
)

results

{'ids': [['id7', 'id4', 'id9', 'id8', 'id6']],
 'embeddings': None,
 'documents': [['Luca Verdi è nato il 12 gennaio 1996 a Firenze, Italia. È alto 1.82 m, ha i capelli corti biondi e occhi azzurri. Attualmente vive a Roma, dove si è trasferito per lavoro dopo aver completato i suoi studi universitari.',
   'Maria Neri è nata il 23 agosto 1998 a Napoli, Italia. È alta 1.65 m, ha capelli neri lunghi e occhi marroni. Attualmente risiede a Torino, dove si è trasferita per continuare i suoi studi e la sua carriera professionale.',
   "Luca ha una passione per lo sport, praticando calcio e tennis fin dalla giovane età. È membro attivo di un club di calcio locale a Roma e partecipa a tornei amatoriali di tennis durante il fine settimana. Oltre allo sport, Luca è un avido lettore di libri di storia e politica, e si dedica al volontariato, collaborando con organizzazioni che promuovono la pace e lo sviluppo internazionale. Ha anche un interesse per la fotografia, che pratica principalmente dur

### Embeddings

di default Chroma utilizza Sentence-Transformers [https://www.sbert.net/](https://www.sbert.net/) come motore di Embeddings.


In [ ]:
from chromadb.utils import embedding_functions
default_ef = embedding_functions.DefaultEmbeddingFunction()

In [ ]:
vect = default_ef(["ciao sono Enzo e sto facendo lezione"])

for elem in vect:
    print(elem, " ", end="")

[-3.40456106e-02  1.42585009e-01 -5.45393415e-02 -4.82498342e-03
 -1.04866482e-01  4.82379831e-03  6.05639704e-02  7.79212341e-02
 -7.58783426e-04  1.80024039e-02  5.16310595e-02 -5.15608080e-02
 -8.78016464e-03  4.92463373e-02 -8.97342619e-03 -1.15944877e-01
  4.02539922e-03  8.45696777e-02 -3.74879390e-02  4.53163646e-02
  5.07213809e-02 -6.80291206e-02 -2.86464021e-02  7.17150718e-02
 -1.04363307e-01  3.78224701e-02  4.66897003e-02  8.09602439e-02
 -3.52062024e-02 -2.34646872e-02 -1.59869865e-02  4.86085974e-02
  8.84322450e-02 -6.29270673e-02 -1.40847713e-02 -5.53731956e-02
 -6.81539997e-03 -5.50173521e-02  2.26212703e-02  2.35559642e-02
 -9.14513916e-02 -1.46434335e-02 -6.03373758e-02  2.76039932e-02
  6.98392168e-02 -2.87992936e-02  9.37215239e-02  8.30309615e-02
  1.09097837e-02 -8.60176906e-02 -4.30417731e-02 -6.77251294e-02
  1.17956903e-02 -1.88096743e-02 -1.19673302e-02  4.28363904e-02
  4.89943800e-03 -5.56470603e-02  2.83171870e-02 -5.81808574e-02
  3.71429026e-02 -4.20907

E' possibile naturalmente integrare Chroma con altri Embeddings, come ad esempio i pricipali disponibili su [OpenAI](https://platform.openai.com/docs/guides/embeddings/embedding-models):
* text-embedding-ada-002
* text-embedding-3-small
* text-embedding-3-large

In [4]:
import os
import chromadb.utils.embedding_functions as embedding_functions

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY"),
    model_name="text-embedding-3-small"
)

In [ ]:
vect = openai_ef(['ciao sono Enzo e sto facendo lezione'])

for elem in vect[0]:
    print(elem, " ", end="")

0.0074030273  -0.0362832  0.019051293  -0.023212882  0.02260642  -0.008715287  -0.040946685  0.06779835  -0.026517058  -0.058638666  0.013206247  -0.006707686  -0.022292731  -0.0013815324  0.032309823  0.04238965  -0.030009447  0.011930585  0.010142566  0.066208996  0.06169189  -0.004807262  -0.0036152494  0.016782286  0.0100746  -0.07737627  0.065707095  0.05307594  0.0027107836  -0.02361022  0.03536305  -0.035697646  0.020222394  -0.01877943  0.023463832  -0.02787637  0.0024219295  0.03515392  -0.0402984  -0.048015114  -0.007068427  -0.006916811  0.028608307  -0.02528322  -0.018873537  -0.032498036  0.02904747  0.012568416  0.07419757  -0.008297036  -0.019804142  0.019595018  0.018099774  0.051695716  -0.0005551617  -0.027980933  0.039608285  0.036847834  -0.048349712  0.05516719  0.025596907  -0.008087912  0.033585485  0.03856266  -0.009337434  0.034087386  -0.02762542  -0.0050085452  -0.007638293  -0.060897216  0.02938207  0.006231927  -0.0014691035  0.023505656  -0.010027546  -0.0

In [ ]:
print(len(vect))

1


In [ ]:
vect = openai_ef(["il machine learning è veramente affascinante, come un po' tutta l'intelligenza artificiale, certo, ma il machine learning, anzi, il deep learning in particolare, ha quella marcia in più..."])
print(len(vect[0]))

1536


In [ ]:
print(len(vect))